In [1]:
import pandas as pd
import numpy as np
import gzip
import os
import json
import time

In [ ]:
'''
Event Stream Data 압축 푼 파일들이 손상되어 있어서 roundstart data가 없는 경우가 있음. 차후 ESD 완전한 상태에서 재수정 할 것.
'''

In [3]:
# set path
path = 'D:/owl-data/Event Stream Data/2020 Season Match Data/'
dir_list = os.listdir(path)

# get folder list
folder_list = []
for f in dir_list:
    if os.path.isdir(path + f):
        folder_list.append(f)
folder_list = folder_list[:-2] # remove 'guids' folder

df1 = pd.DataFrame()
df2 = pd.DataFrame()
for f in folder_list:
    file_list = os.listdir(path + f)
    file_list = [file for file in file_list if file.endswith('.gz')] # get gz files only
    roundstart_file_name = [g for g in file_list if 'roundstart' in g] # return gamestart file names
    gameresult_file_name = [g for g in file_list if 'gameresult' in g] # return gamestart file names
    print(roundstart_file_name)
    print(gameresult_file_name)

    df_roundstart = pd.read_csv(os.path.join(path, f, roundstart_file_name[0]), delimiter = '\t', compression='gzip')
    df1 = pd.concat([df1, df_roundstart], axis=0, ignore_index=True) # concat data info df
    df_gameresult = pd.read_csv(os.path.join(path, f, gameresult_file_name[0]), delimiter = '\t', compression='gzip')
    df2 = pd.concat([df2, df_gameresult], axis=0, ignore_index=True) # concat data info df
display(df1)
display(df2)

['payload_roundstart-20200622.tsv.gz']
['payload_gameresult-20200622.tsv.gz']


EmptyDataError: No columns to parse from file

In [ ]:
# add map_guid column in gameresult df
def get_map_guid(str):
    '''
    args, json text from gameresult
    target, map_guid
    '''
    jsontxt = json.loads(str)
    map_guid = jsontxt['game_context']['map_guid']
    
    return map_guid

map_guid_list = []
for idx in df2.index:
    map_guid = get_map_guid(df2.loc[idx, 'info'])
    map_guid_list.append(map_guid)

df2['map_guid'] = map_guid_list
df2 = df2[df2['end_reason'] == 'NORMAL']
display(df2)

In [ ]:
# merge df1(gamestart) and df2(gameresult)
df_merge = pd.merge(df1, df2, how='left', left_on=['map_guid', 'esports_match_id'], right_on=['map_guid', 'esports_match_id'], suffixes=('_gamestart', '_gameresult'))
df_merge = df_merge[['time_gamestart', 'map_guid', 'esports_match_id', 'time_gameresult']]
display(df_merge)

In [ ]:
# map_guid mapping
map_guid = pd.read_csv((os.path.join('D:\owl-data\guid', 'map_guid.csv')), delimiter=',')
display(map_guid)
df_ = pd.merge(df_merge, map_guid, how='left', left_on='map_guid', right_on='guid')
display(df_)

In [ ]:
# save df to pickle
df_ = df_[['time_gamestart', 'time_gameresult', 'esports_match_id', 'en_map_name', 'map_guid']]
display(df_)
df_.to_pickle(os.path.join('D:\owl-data\pickles', 'OWL_map_start_info'))

In [ ]:
# read pickle
test = pd.read_pickle(os.path.join('D:\owl-data\pickles', 'OWL_map_start_info'))
display(test)